<a href="https://colab.research.google.com/github/ChrizzBln89/ChrizzBln89/blob/main/PPO_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

# --- 1. Google Drive verbinden ---
try:
    drive.mount('/content/drive/')
    print("Google Drive erfolgreich verbunden.")
except Exception as e:
    print(f"Fehler beim Verbinden mit Google Drive: {e}")

# --- 2. Pfade definieren ---
BASE_DIR = '/content/drive/MyDrive/data/PPO_portfolio_optimization'
INPUT_CSV = os.path.join(BASE_DIR, 'sp500_20_years_data.csv')
OUTPUT_NPY = os.path.join(BASE_DIR, 'sp500_features_V1.npy')

print(f"Lade Rohdaten von: {INPUT_CSV}")

# --- 3. Rohdaten laden ---
try:
    # header=[0, 1] für MultiIndex-Spalten (z.B. 'Adj Close' -> 'AAPL')
    data = pd.read_csv(INPUT_CSV, header=[0, 1], index_col=0, parse_dates=True)
    print("Rohdaten erfolgreich geladen.")

    # --- 4. Ticker und Daten extrahieren ---
    adj_close = data['Adj Close']
    volume = data['Volume']

    # Bereinigung: Entferne den S&P 500 Index ('^GSPC')
    if '^GSPC' in adj_close.columns:
        adj_close = adj_close.drop(columns='^GSPC')
    if '^GSPC' in volume.columns:
        volume = volume.drop(columns='^GSPC')

    # Sicherstellen, dass die Spalten übereinstimmen
    shared_tickers = adj_close.columns.intersection(volume.columns)
    adj_close = adj_close[shared_tickers]
    volume = volume[shared_tickers]

    print(f"Verarbeite {len(shared_tickers)} Ticker (nach Bereinigung).")

    # --- 5. Features berechnen ---
    # (DataFrames behalten den Zeitindex und die Ticker-Namen)

    # Feature 1: Log Returns
    f1_log_returns = np.log(adj_close / adj_close.shift(1))

    # Feature 2: Volatilität (Rollierende 20-Tage-Standardabweichung)
    f2_volatility = f1_log_returns.rolling(window=20).std()

    # Feature 3: Volumen-Verhältnis (Normalisiertes Volumen)
    volume_rolling_mean = volume.rolling(window=20).mean()
    f3_volume_ratio = volume / volume_rolling_mean

    print("Features berechnet: Log Returns, Volatilität, Volumen-Verhältnis.")

    # --- 6. 3D-Tensor erstellen ---
    # Liste unserer 2D-Feature-DataFrames
    features_list = [f1_log_returns, f2_volatility, f3_volume_ratio]

    # Wandle DataFrames in NumPy-Arrays um (.values) und staple sie
    # entlang einer neuen dritten Achse (axis=2).
    # Das Ergebnis hat die Form (Tage, Assets, Features)
    feature_tensor_3d = np.stack([f.values for f in features_list], axis=2)

    # --- 7. NaNs (Not a Number) bereinigen ---
    # Ersetze NaNs (von rolling/shift) und unendliche Werte durch 0.0
    feature_tensor_3d = np.nan_to_num(feature_tensor_3d, nan=0.0, posinf=0.0, neginf=0.0)

    print(f"3D-Tensor erfolgreich erstellt.")
    print(f"Finale Tensor-Form: {feature_tensor_3d.shape}")
    print("(Tage, Assets, Features)")

    # --- 8. 3D-Tensor als .npy speichern ---
    np.save(OUTPUT_NPY, feature_tensor_3d)

    print(f"--- ERFOLG ---")
    print(f"Feature-Tensor gespeichert unter: {OUTPUT_NPY}")

except FileNotFoundError:
    print(f"FEHLER: Die Datei {INPUT_CSV} wurde nicht gefunden.")
except Exception as e:
    print(f"Ein unerwarteter Fehler ist aufgetreten: {e}")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Google Drive erfolgreich verbunden.
Lade Rohdaten von: /content/drive/MyDrive/data/PPO_portfolio_optimization/sp500_20_years_data.csv
Rohdaten erfolgreich geladen.
Ein unerwarteter Fehler ist aufgetreten: 'Adj Close'
